In [1]:
import pandas as pd
import re

from nltk.corpus import stopwords
stop_words = list(set(stopwords.words('english')))
stop_words.extend(['from', 'subject', 're', 'edu', 'use','http'])

from textblob import TextBlob


In [2]:
# Load Data
path = 'D:/JOB PREP/Projects/Data Science at British Airways/'
df = pd.read_csv(path+"data/BA_reviews.csv")

### Preprocessing / Data Cleaning

In [3]:
#lowercase & puntuation
def lowerPunc(element):
    s = ''.join([char for char in element.lower() if char.isalpha() or char.isspace()])
    return s

#remove stopwords
def remove_stopwords(text):
    return " ".join([word for word in str(text).split() if word not in stop_words])


In [4]:
import nltk
def data_cleaning(df,var):
    # Removed futile text from the data
    df = df[var].str.split('|', 1, expand=True).rename(columns={1:var})
    df[var] = df[var].fillna(df[0])
    df = df.drop(0,axis=1)
    
    #lowercase & puntuation
    df[var] = df[var].apply(lowerPunc)
    
    #remove stopwords
    df[var] = df[var].apply(lambda x: remove_stopwords(x))

    # Remove extra space
    df[var] = df[var].apply(lambda text: re.sub(' +', ' ', text))
    
    return df

df = data_cleaning(df,'reviews')
df.head()

C:\Users\anjal\AppData\Local\Temp\ipykernel_20632\296908727.py:4: FutureWarning: In a future version of pandas all arguments of StringMethods.split except for the argument 'pat' will be keyword-only.
  df = df[var].str.split('|', 1, expand=True).rename(columns={1:var})


,reviews
0,worst cabin experience ever cramped seats low ...
1,first time flying british airways first time f...
2,latest affront stood first class line hour fin...
3,booked flight return flight london munich dece...
4,tried check line informed possible given reaso...


### Sentiment Analysis

In [5]:
def sentiment(text):
    senti =TextBlob(text)
    return senti.sentiment

df['polarity']=df['reviews'].apply(sentiment).apply(lambda b: b[0])

df.head(10)

,reviews,polarity
0,worst cabin experience ever cramped seats low ...,-0.380000
1,first time flying british airways first time f...,0.176719
2,latest affront stood first class line hour fin...,0.182857
3,booked flight return flight london munich dece...,0.062500
4,tried check line informed possible given reaso...,-0.083333
5,catalogue failures flew belfast heathrow week ...,-0.033333
6,flight dubrovnik painful time morning london g...,0.143747
7,could give minus rating would supposed lhrams ...,0.066667
8,plane hour late leaving problem happen arrival...,0.166667
9,flying world traveller plus premium service pr...,0.032222


In [6]:
# identify category
def categories(pol):
    if pol>0:
        return 'positive'
    elif pol==0:
        return 'neutral'
    else:
        return 'negative'
    
df['review_sentiment']=df['polarity'].apply(categories)
df.head(10)

,reviews,polarity,review_sentiment
0,worst cabin experience ever cramped seats low ...,-0.380000,negative
1,first time flying british airways first time f...,0.176719,positive
2,latest affront stood first class line hour fin...,0.182857,positive
3,booked flight return flight london munich dece...,0.062500,positive
4,tried check line informed possible given reaso...,-0.083333,negative
5,catalogue failures flew belfast heathrow week ...,-0.033333,negative
6,flight dubrovnik painful time morning london g...,0.143747,positive
7,could give minus rating would supposed lhrams ...,0.066667,positive
8,plane hour late leaving problem happen arrival...,0.166667,positive
9,flying world traveller plus premium service pr...,0.032222,positive


In [7]:
df['review_sentiment'].value_counts()

positive    2503
negative     970
neutral       22
Name: review_sentiment, dtype: int64

In [8]:
df['review_sentiment'].value_counts(normalize=True)

positive    0.716166
negative    0.277539
neutral     0.006295
Name: review_sentiment, dtype: float64